In [1]:
from genson import SchemaBuilder
import ccxt
from copy import deepcopy
import uuid
from random import randint, sample

In [2]:
def get_uuids(vals: int):
    return [uuid.uuid4().hex for _ in range(vals)]

In [3]:
fid = uuid.uuid4().hex
existing_ids = get_uuids(30)

In [4]:
nested_pids = sample(existing_ids, 7)

In [5]:
nested_set = {k: get_uuids(randint(4, 12))for k in nested_pids}

In [6]:
# {fid: existing_ids, **nested_set}

In [7]:
print("Hello world")

Hello world


In [8]:
binance = ccxt.binance()
markets = binance.load_markets()
# Requisite setup step


In [9]:
import json
from datetime import datetime
from pydantic import BaseModel, Field
from typing import List
import random
import httpx

In [10]:
class Candle(BaseModel):
    _open: Field(..., alias ="open")
    high: float
    low: float
    close: float
    volume: float
    timestamp: datetime

In [11]:
class Candles(BaseModel):
    candles: List[Candle] = []

In [12]:
pair = 'ETH/USDT'
ohlcv = binance.fetch_ohlcv(pair, '1d')
candles = [Candle(timestamp=candle[0], open=candle[1], high=candle[2], low=candle[3], close=candle[4],volume=candle[-1]) for candle in ohlcv]
candy = Candles(candles=candles)

In [13]:
random.choice(candy.candles).dict()

{'high': 127.84,
 'low': 123.07,
 'close': 125.09,
 'volume': 225004.4909,
 'timestamp': datetime.datetime(2019, 12, 25, 0, 0, tzinfo=datetime.timezone.utc)}

In [14]:
import orjson

In [57]:
evnt_at = random.choice(candy.candles)
resp = httpx.post('http://127.0.0.1:8000/record/insert', content=orjson.dumps({"data": evnt_at.dict(), "bucket": "super_candle", "event_at": evnt_at.timestamp, "tags": {"hello": "world"}}).decode())

In [43]:
# json.dumps(ohlcv)

In [44]:
# straight_market_builder = SchemaBuilder()
# straight_market_builder.add_object(test_market)

In [45]:
# market_builder = SchemaBuilder()
# market_builder.add_object(test_market_removed)

In [46]:
# market_builder.add_object(test_market)
# market_builder.to_json()

In [47]:
# straight_market_builder.to_schema()

In [48]:
# market_schema = market_builder.to_schema()

In [49]:
# market_schema

In [50]:
# builder = SchemaBuilder()
# builder.add_schema({"type": "object", "properties": {}})
# builder.add_object({"hi": "there"})
# builder.add_object({"hi": 5})
# builder.to_schema()

In [51]:
# b1 = SchemaBuilder()
# b2 = SchemaBuilder()
# b1.add_schema({"type": "object", "properties": {"hi": {"type": "string"}}})
# b2.add_schema({"type": "object", "properties": {"hi": {"type": "integer"}}})
# b1 == b2

In [52]:
# b1.add_schema(b2)
# b2.add_schema(b1)

In [15]:
# b1 == b2

In [5]:
import uuid
from datetime import datetime
from mangostar.clients.faas import FaaSClient

In [6]:
faas_cli = FaaSClient()

In [10]:
cake_item = {
    "bucket": "bakery_inventory",
    "data": {
	"id": "0001",
	"type": "donut",
	"name": "Cake",
	"ppu": 0.55,
	"batters":
		{
			"batter":
				[
					{ "id": "1001", "type": "Regular" },
					{ "id": "1002", "type": "Chocolate" },
					{ "id": "1003", "type": "Blueberry" },
					{ "id": "1004", "type": "Devil's Food" }
				]
		},
	"topping":
		[
			{ "id": "5001", "type": "None" },
			{ "id": "5002", "type": "Glazed" },
			{ "id": "5005", "type": "Sugar" },
			{ "id": "5007", "type": "Powdered Sugar" },
			{ "id": "5006", "type": "Chocolate with Sprinkles" },
			{ "id": "5003", "type": "Chocolate" },
			{ "id": "5004", "type": "Maple" }
		]
},
"event_at": datetime.now(),
"tags": {"store_name": "shoeberry bakery", "city": "los angeles", "state": "california", "id": uuid.uuid4().hex}
}



In [13]:
from pydantic import BaseModel
from typing import Dict

In [20]:
class RequestModel(BaseModel):
    data: Dict

In [22]:
from datetime import datetime
from typing import Any, Dict, Optional

from addict import Addict as DDict
from fastapi import FastAPI, HTTPException
from inflection import tableize
from loguru import logger
from mangostar import commands as comms
from mangostar.circular import DBResponse
# from mangostar.mangostar import commands as comm
from pydantic import BaseModel, root_validator


In [27]:
class InsertableRecord(BaseModel):
    data: Dict[str, Any]
    event_at: datetime = datetime.now()
    bucket: Optional[str]
    tags: Dict[str, Any] = {}

    @root_validator
    def check_bucket(cls, values: dict):
        doti = DDict(**values)
        buck = doti.bucket
        if buck:
            doti.bucket = tableize(buck)
            return doti.to_dict()

        if "bucket" not in doti.data.keys():
            raise ValueError("Bucket not added.")

        nested_bucket = doti.data.bucket
        if not isinstance(nested_bucket, str):
            raise TypeError(
                "A bucket was found in 'data' but it wasn't the right type.")
        doti.data.pop("bucket", None)

        doti.bucket = str(nested_bucket)
        return doti.to_dict()

In [8]:
faas_cli(data=cake_item, name="poopyy")

<Response [500 Internal Server Error]>

In [22]:
resp:str = orjson.dumps(cake_item).decode('utf-8')



open("", "+w")

{"bucket":"bakery_inventory","data":{"id":"0001","type":"donut","name":"Cake","ppu":0.55,"batters":{"batter":[{"id":"1001","type":"Regular"},{"id":"1002","type":"Chocolate"},{"id":"1003","type":"Blueberry"},{"id":"1004","type":"Devil's Food"}]},"topping":[{"id":"5001","type":"None"},{"id":"5002","type":"Glazed"},{"id":"5005","type":"Sugar"},{"id":"5007","type":"Powdered Sugar"},{"id":"5006","type":"Chocolate with Sprinkles"},{"id":"5003","type":"Chocolate"},{"id":"5004","type":"Maple"}]},"event_at":"2021-03-19T20:55:31.193031","tags":{"store_name":"shoeberry bakery","city":"los angeles","state":"california","id":"7a1dc891b61649f59448cc02ba586278"}}


In [ ]:
resp:str = orjson.dumps(cake_item).decode('utf-8')

print(resp)

In [16]:
from mangostar import commands as comm
from datetime import datetime
import uuid
cake_item = {
    "bucket": "bakery_inventory",
    "data": {
    "id": "0001",
    "type": "donut",
    "name": "Cake",
    "ppu": 0.55,
    "batters":
            {
                    "batter":
                            [
                                    { "id": "1001", "type": "Regular" },
                                    { "id": "1002", "type": "Chocolate" },
                                    { "id": "1003", "type": "Blueberry" },
                                    { "id": "1004", "type": "Devil's Food" }
                            ]
            },
        "topping":
            [
                    { "id": "5001", "type": "None" },
                    { "id": "5002", "type": "Glazed" },
                    { "id": "5005", "type": "Sugar" },
                    { "id": "5007", "type": "Powdered Sugar" },
                    { "id": "5006", "type": "Chocolate with Sprinkles" },
                    { "id": "5003", "type": "Chocolate" },
                    { "id": "5004", "type": "Maple" }
            ]
        },
        "event_at": datetime.now(),
        "tags": {"store_name": "shoeberry bakery", "city": "los angeles", "state": "california", "id": uuid.uuid4().hex}
}

In [24]:
req = RequestModel(data=cake_item)

In [30]:
req.data

{'bucket': 'bakery_inventory',
 'data': {'id': '0001',
  'type': 'donut',
  'name': 'Cake',
  'ppu': 0.55,
  'batters': {'batter': [{'id': '1001', 'type': 'Regular'},
    {'id': '1002', 'type': 'Chocolate'},
    {'id': '1003', 'type': 'Blueberry'},
    {'id': '1004', 'type': "Devil's Food"}]},
  'topping': [{'id': '5001', 'type': 'None'},
   {'id': '5002', 'type': 'Glazed'},
   {'id': '5005', 'type': 'Sugar'},
   {'id': '5007', 'type': 'Powdered Sugar'},
   {'id': '5006', 'type': 'Chocolate with Sprinkles'},
   {'id': '5003', 'type': 'Chocolate'},
   {'id': '5004', 'type': 'Maple'}]},
 'event_at': datetime.datetime(2021, 3, 20, 14, 13, 32, 647050),
 'tags': {'store_name': 'shoeberry bakery',
  'city': 'los angeles',
  'state': 'california',
  'id': '679861c10188459c89547ee2ee973790'}}

In [31]:
import orjson

In [34]:
insertable = InsertableRecord(**orjson.loads(orjson.dumps(req.data)))

In [35]:
comm.insert_dict(**insertable.dict())

DBResponse(status=False, data={}, message='Failed to add record somewhere along the way.')